## 可運行的 DSPy

In [1]:
# install DSPy: pip install dspy
import dspy

# Ollam is now compatible with OpenAI APIs
# 
# To get this to work you must include `model_type='chat'` in the `dspy.OpenAI` call. 
# If you do not include this you will get an error. 
# 
# I have also found that `stop='\n\n'` is required to get the model to stop generating text after the ansewr is complete. 
# At least with mistral.

ollama_model = dspy.OpenAI(api_base='http://localhost:11434/v1/', api_key='ollama', model='llama3', stop='\n\n', model_type='chat')

# This sets the language model for DSPy.
dspy.settings.configure(lm=ollama_model)

# This is not required but it helps to understand what is happening
my_example = {
    "question": "What game was Super Mario Bros. 2 based on?",
    "answer": "Doki Doki Panic",
}

# This is the signature for the predictor. It is a simple question and answer model.
class BasicQA(dspy.Signature):
    """Answer questions about classic video games."""

    question = dspy.InputField(desc="a question about classic video games")
    answer = dspy.OutputField(desc="often between 1 and 5 words")

# Define the predictor.
generate_answer = dspy.Predict(BasicQA)

# Call the predictor on a particular input.
pred = generate_answer(question=my_example['question'])

# Print the answer...profit :)
print(pred.answer)

Doki Doko Panic


## YT demo code

In [30]:
!pip install rich

/Users/matt_huang/.pyenv/versions/3.12.0/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=24657) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


In [2]:
import sys
import os
import dspy
from dspy.datasets import HotPotQA
from dspy.teleprompt import BootstrapFewShot
from dspy.evaluate.evaluate import Evaluate
from dsp.utils import deduplicate
from rich import print

### 1. Configuration & Data Loading

In [3]:
# turbo = dspy.OpenAI(model='gpt-3.5-turbo')
ollama_model = dspy.OpenAI(api_base='http://localhost:11434/v1/', api_key='ollama', model='llama3', stop='\n\n', model_type='chat')
colbertv2_wiki17_abstracts = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')
dspy.settings.configure(lm=ollama_model, rm=colbertv2_wiki17_abstracts)
dataset = HotPotQA(train_seed=1, train_size=20, eval_seed=2023, dev_size=50, test_size=0)

trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]

print(len(trainset), len(devset))
print(f"Trainset Data {trainset[:5]}")
print(f"Devset Data {devset[:5]}")

print("\n### Example Question with Answer ###\n")
example = devset[18]
print(f"Question: {example.question}")
print(f"Answer: {example.answer}")
print(f"Relevant Wikipedia Titles: {example.gold_titles}")

/Users/matt_huang/Desktop/dev/bootcamp_ai/venv/lib/python3.12/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


20 50

Trainset Data [Example({'question': 'At My Window was released by which American singer-songwriter?', 'answer': 
'John Townes Van Zandt'}) (input_keys={'question'}), Example({'question': 'which  American actor was Candace Kita  
guest starred with ', 'answer': 'Bill Murray'}) (input_keys={'question'}), Example({'question': 'Which of these 
publications was most recently published, Who Put the Bomp or Self?', 'answer': 'Self'}) (input_keys={'question'}),
Example({'question': 'The Victorians - Their Story In Pictures is a documentary series written by an author born in
what year?', 'answer': '1950'}) (input_keys={'question'}), Example({'question': 'Which magazine has published 
articles by Scott Shaw, Tae Kwon Do Times or Southwest Art?', 'answer': 'Tae Kwon Do Times'}) 
(input_keys={'question'})]

Devset Data [Example({'question': 'Are both Cangzhou and Qionghai in the Hebei province of China?', 'answer': 'no',
'gold_titles': {'Qionghai', 'Cangzhou'}}) (input_keys={'question'}), Example({'question': 'Who conducts the draft 
in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?', 'answer': 'National 
Hockey League', 'gold_titles': {'2017–18 Pittsburgh Penguins season', '2017 NHL Expansion Draft'}}) 
(input_keys={'question'}), Example({'question': 'The Wings entered a new era, following the retirement of which 
Canadian retired professional ice hockey player and current general manager of the Tampa Bay Lightning of the 
National Hockey League (NHL)?', 'answer': 'Steve Yzerman', 'gold_titles': {'Steve Yzerman', '2006–07 Detroit Red 
Wings season'}}) (input_keys={'question'}), Example({'question': 'What river is near the Crichton Collegiate 
Church?', 'answer': 'the River Tyne', 'gold_titles': {'Crichton Collegiate Church', 'Crichton Castle'}}) 
(input_keys={'question'}), Example({'question': 'In the 10th Century A.D. Ealhswith had a son called Æthelweard by 
which English king?', 'answer': 'King Alfred the Great', 'gold_titles': {'Æthelweard (son of Alfred)', 
'Ealhswith'}}) (input_keys={'question'})]

### Example Question with Answer ###

Question: What is the nationality of the chef and restaurateur featured in Restaurant: Impossible?

Answer: English

Relevant Wikipedia Titles: {'Restaurant: Impossible', 'Robert Irvine'}

### 2. Basic Chatbot

In [33]:
class BasicQA(dspy.Signature):  # A. Signature
    """Answer questions with short factoid answers."""
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

print("\n### Generate Response ###\n")
generate_answer = dspy.Predict(BasicQA)
pred = generate_answer(question=example.question)
print(f"Question: {example.question}\nPredicted Answer: {pred.answer}")

### Generate Response ###

Question: What is the nationality of the chef and restaurateur featured in Restaurant: Impossible?
Predicted Answer: Robert Irvine

### 3. Chatbot with Chain of Thought

In [34]:
print("\n### Generate Response with Chain of Thought ###\n")
generate_answer_with_chain_of_thought = dspy.ChainOfThought(BasicQA)
pred = generate_answer_with_chain_of_thought(question=example.question)
print(f"Question: {example.question}\nThought: {pred.rationale.split('.', 1)[1].strip()}\nPredicted Answer: {pred.answer}")

### Generate Response with Chain of Thought ###

Question: What is the nationality of the chef and restaurateur featured in Restaurant: Impossible?
Thought: 
Predicted Answer: British

### 4. Chatbot with Chain of Thought and Context = RAG --> (Retrieve, Generate Response)

In [35]:
print("\n### RAG: Generate Response with Chain of Thought and Context ###\n")

# 4a. Signature
class GenerateAnswer(dspy.Signature): 
    """Answer questions with short factoid answers."""
    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

# 4b. Module / Pipeline
class RAG(dspy.Module): 
    def __init__(self, num_passages=3):
        super().__init__()
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
    
    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

# 4c. Optimizer / Optimising Pipeline
def validate_context_and_answer(example, pred, trace=None): 
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM and answer_PM

teleprompter = BootstrapFewShot(metric=validate_context_and_answer) 
compiled_rag = teleprompter.compile(RAG(), trainset=trainset)

# 4d. Executing Pipeline
my_question = "What castle did David Gregory inherit?" 
pred = compiled_rag(my_question)

print(f"Question: {my_question}")
print(f"Predicted Answer: {pred.answer}")
print(f"Retrieved Contexts (truncated): {[c[:200] + '...' for c in pred.context]}")

### RAG: Generate Response with Chain of Thought and Context ###

100%|████████████████████████████████████████████████| 20/20 [27:16<00:00, 81.85s/it]


Bootstrapped 0 full traces after 20 examples in round 0.


Question: What castle did David Gregory inherit?

Predicted Answer: Context: [1

Retrieved Contexts (truncated): ['David Gregory (physician) | David Gregory (20 December 1625 – 1720) was a 
Scottish physician and inventor. His surname is sometimes spelt as Gregorie, the original Scottish spelling. He 
inherited Kinn...', 'Gregory Tarchaneiotes | Gregory Tarchaneiotes (Greek: Γρηγόριος Ταρχανειώτης , Italian: 
"Gregorio Tracanioto" or "Tracamoto" ) was a "protospatharius" and the long-reigning catepan of Italy from 998 
t...', 'David Gregory (mathematician) | David Gregory (originally spelt Gregorie) FRS (? 1659 – 10 October 1708) 
was a Scottish mathematician and astronomer. He was professor of mathematics at the University ...']

### 5. Evaluating the Answers

In [39]:
print("\n### Evaluating the Answers ###\n")

# 5a. Basic RAG
def gold_passages_retrieved(example, pred, trace=None):
    gold_titles = set(map(dspy.evaluate.normalize_text, example['gold_titles']))
    found_titles = set(map(dspy.evaluate.normalize_text, [c.split(' | ')[0] for c in pred.context]))
    return gold_titles.issubset(found_titles)

evaluate_on_hotpotqa = Evaluate(devset=devset, num_threads=1, display_progress=True, display_table=5)
compiled_rag_retrieval_score = evaluate_on_hotpotqa(compiled_rag, metric=gold_passages_retrieved)

# 5b. Uncompiled Baleen RAG (without Optimizer)
class GenerateSearchQuery(dspy.Signature):
    """Write a simple search query that will help answer a complex question."""
    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    query = dspy.OutputField()

class SimplifiedBaleen(dspy.Module):
    def __init__(self, passages_per_hop=3, max_hops=2):
        super().__init__()
        self.generate_query = [dspy.ChainOfThought(GenerateSearchQuery) for _ in range(max_hops)]
        self.retrieve = dspy.Retrieve(k=passages_per_hop)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
        self.max_hops = max_hops
    
    def forward(self, question):
        context = []
        for hop in range(self.max_hops):
            query = self.generate_query[hop](context=context, question=question).query
            passages = self.retrieve(query).passages
            context = deduplicate(context + passages)
        pred = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=pred.answer)

uncompiled_baleen = SimplifiedBaleen()  # uncompiled (i.e., zero-shot) program
pred = uncompiled_baleen(my_question)
print(f"Question: {my_question}")
print(f"Predicted Answer: {pred.answer}")
print(f"Retrieved Contexts (truncated): {[c[:200] + '...' for c in pred.context]}")

# 5c. Compiled Baleen RAG (with Optimizer)
def validate_context_and_answer_and_hops(example, pred, trace=None):
    if not dspy.evaluate.answer_exact_match(example, pred): return False
    if not dspy.evaluate.answer_passage_match(example, pred): return False
    hops = [example.question] + [outputs.query for *_, outputs in trace if 'query' in outputs]
    if max([len(h) for h in hops]) > 100: return False
    if any(dspy.evaluate.answer_exact_match_str(hops[idx], hops[:idx], frac=0.8) for idx in range(2, len(hops))): return False
    return True

teleprompter = BootstrapFewShot(metric=validate_context_and_answer_and_hops)
compiled_baleen = teleprompter.compile(SimplifiedBaleen(), teacher=SimplifiedBaleen(passages_per_hop=2), trainset=trainset)
uncompiled_baleen_retrieval_score = evaluate_on_hotpotqa(uncompiled_baleen, metric=gold_passages_retrieved)
compiled_baleen_retrieval_score = evaluate_on_hotpotqa(compiled_baleen, metric=gold_passages_retrieved)

print(f"## Retrieval Score for RAG: {compiled_rag_retrieval_score}")
print(f"## Retrieval Score for uncompiled Baleen: {uncompiled_baleen_retrieval_score}")
print(f"## Retrieval Score for compiled Baleen: {compiled_baleen_retrieval_score}")

compiled_baleen("How many storeys are in the castle that David Gregory inherited?")

# ollama_model.inspect_history(n=1)
# ollama_model.inspect_history(n=3)

### Evaluating the Answers ###



  0%|                                                         | 0/50 [00:00<?, ?it/s]

Average Metric: 0 / 1  (0.0):   0%|                           | 0/50 [00:00<?, ?it/s]

Average Metric: 1 / 2  (50.0):   2%|▎                 | 1/50 [00:00<00:01, 35.77it/s]

Average Metric: 2 / 3  (66.7):   4%|▋                 | 2/50 [00:00<00:00, 48.94it/s]

Average Metric: 3 / 4  (75.0):   6%|█                 | 3/50 [00:00<00:00, 57.94it/s]

Average Metric: 3 / 5  (60.0):   8%|█▍                | 4/50 [00:00<00:00, 65.83it/s]

Average Metric: 4 / 6  (66.7):  10%|█▊                | 5/50 [00:00<00:00, 65.08it/s]

Average Metric: 4 / 7  (57.1):  12%|██▏               | 6/50 [00:00<00:00, 67.52it/s]

Average Metric: 5 / 8  (62.5):  14%|██▌               | 7/50 [00:00<00:00, 72.05it/s]

Average Metric: 5 / 8  (62.5):  16%|██▉               | 8/50 [00:00<00:00, 78.82it/s]

Average Metric: 5 / 9  (55.6):  16%|██▉               | 8/50 [00:00<00:00, 78.82it/s]

Average Metric: 5 / 10  (50.0):  18%|███ 

Average Metric: 13 / 50  (26.0%)


,question,example_answer,gold_titles,context,pred_answer,gold_passages_retrieved
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Cangzhou', 'Qionghai'}","['Cangzhou | Cangzhou () is a prefecture-level city in eastern Hebei province, People\'s Republic of China. At the 2010 census, Cangzhou\'s built-up (""or metro"") area...",Context,False
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017 NHL Expansion Draft', '2017–18 Pittsburgh Penguins season'}",['2017–18 Pittsburgh Penguins season | The 2017–18 Pittsburgh Penguins season will be the 51st season for the National Hockey League ice hockey team that was...,Context,✔️ [True]
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'Steve Yzerman', '2006–07 Detroit Red Wings season'}","['Steve Yzerman | Stephen Gregory ""Steve"" Yzerman ( ; born May 9, 1965) is a Canadian retired professional ice hockey player and current general manager...",Context,✔️ [True]
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Collegiate Church', 'Crichton Castle'}","[""Crichton Collegiate Church | Crichton Collegiate Church is situated about 0.6 mi south west of the hamlet of Crichton in Midlothian, Scotland. Crichton itself is...",Context,✔️ [True]
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Æthelweard (son of Alfred)', 'Ealhswith'}","[""Æthelweard of East Anglia | Æthelweard (died 854) was a 9th-century king of East Anglia, the long-lived Anglo-Saxon kingdom which today includes the English counties...",Context: [2] «Æthelweard (son of Alfred) | Æthelweard (d. 920 or 922) was the younger son of King Alfred the Great and Ealhswith,False


Question: What castle did David Gregory inherit?

...

,question,example_answer,gold_titles,context,pred_answer,gold_passages_retrieved
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Cangzhou', 'Qionghai'}",['Complete (The Smiths album) | Complete is a box set released by British band The Smiths on 26 September 2011. Standard versions contain their four...,Context,False
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017 NHL Expansion Draft', '2017–18 Pittsburgh Penguins season'}",['Complete (The Smiths album) | Complete is a box set released by British band The Smiths on 26 September 2011. Standard versions contain their four...,Context,False
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'Steve Yzerman', '2006–07 Detroit Red Wings season'}","[""Here's the Answer | Here's the Answer is the second studio album by American country artist Skeeter Davis. The album was released in January 1961...",Context,False
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Collegiate Church', 'Crichton Castle'}",['Complete (The Smiths album) | Complete is a box set released by British band The Smiths on 26 September 2011. Standard versions contain their four...,"Context: The Crichton Collegiate Church is a 16th-century church located in Edinburgh, Scotland",False
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Æthelweard (son of Alfred)', 'Ealhswith'}","[""Here's the Answer | Here's the Answer is the second studio album by American country artist Skeeter Davis. The album was released in January 1961...",Context,False


## Retrieval Score for RAG: 26.0

## Retrieval Score for uncompiled Baleen: 0.0

## Retrieval Score for compiled Baleen: 0.0

Prediction(
    context=['Complete (The Smiths album) | Complete is a box set released by British band The Smiths on 26 September 2011. Standard versions contain their four studio albums ("The Smiths", "Meat Is Murder", "The Queen Is Dead" and "Strangeways, Here We Come"), a live album ("Rank") and three compilations ("Hatful of Hollow", "The World Won\'t Listen" and "Louder Than Bombs") over 8 CDs or 8 LPs. A deluxe version contains the albums on both CD and LP formats as well as 25 7" vinyl singles and a DVD.', "The Complete Works (Queen album) | The Complete Works is a boxed set issued by the rock band Queen in 1985. It contained all of the band's original studio albums, live album and non-album tracks to that point. It was available in vinyl and cassette formats.", 'The Complete History (album) | The Complete History is a compilation album released by 2 Unlimited on February 9, 2004 through ZYX Music label The record was released in two formats: as a single CD edition, and as a CD/

### quick test for DSPy

In [4]:
import dspy
import time
import ollama

In [5]:
class _basicQA(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.Predict("question -> answer")

    def forward(self, question):
        return self.prog(question = question)

In [22]:
ollama_model.inspect_history(n=5)





Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Answer: ${answer}

---

Question: How many legs does elephant has?
Answer: Question: How many legs does an elephant have?
Answer: 4







Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Answer: ${answer}

---

Question: 什麼是 DSPy 的 Signature?
Answer: Here is the output







Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Answer: ${answer}

---

Question: How many legs does elephant has
Answer: Question: How many legs does an elephant have?
Answer: 4







Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Answer: ${answer}

---

Question: How many legs does bird has
Answer: Question: How many legs does a bird have?
Answer: 2





In [33]:
predictor = dspy.Predict("question->answer")
print(predictor)
print("\n")
print("-"*50)
print("\n")
resp = predictor(question="Hi")
print(resp.answer)

Predict(StringSignature(question -> answer
    instructions='Given the fields `question`, produce the fields `answer`.'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 
'Question:', 'desc': '${question}'})
    answer = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 
'Answer:', 'desc': '${answer}'})
))

--------------------------------------------------

Here is the output

In [28]:
time_stamp1 = time.time()
_Query = "How many legs does bird has"
desc_ans = "often between 1 and 5 words"
_QA_ = ollama.generate(
    model="llama3",
    prompt=f"""
    Answer questions with shrot factoid answers.
    
    ---
    
    Follow the following format.
    
    Question: ${{questino}}
    Question's Answer: {desc_ans}
    
    ---
    Question: {_Query}"""
)
time_stamp2 = time.time()
time_pass1 = (time_stamp2-time_stamp1)*1000
print(f"無調整:\n {_QA_['response']}\n\n費時: {time_pass1:.2f} ms")
print("-"*40)
time_stamp3 = time.time()
_QA = _basicQA()
_resp = _QA.forward(_Query)
time_stamp4 = time.time()
time_pass2 = (time_stamp4-time_stamp3)*1000
print(f"經過 DSPy:\n {_resp.answer}\n\n費時: {time_pass2:.2f} ms")

無調整:
 Question: How many legs does a bird have?
Answer: Two

費時: 2745.11 ms

----------------------------------------

經過 DSPy:
 Question: How many legs does a bird have?
Answer: 2

費時: 3.83 ms

In [41]:
from dspy.retrieve.chromadb_rm import ChromadbRM
retrieve_model = ChromadbRM(
    'docs',
    './db',
    embedding_function='llama3',
    k=5,
)